In [1]:
import sys
sys.path.append('../')
from ship import ShipGenerator
import tensorflow as tf
import numpy as np

In [ ]:
file_dir='/home/minjun/Jupyter/Ship_Detection/Data/train_tfrecorder/train_data.tfrecords'
common_args = {
    'batch_size': 1,
    'phi': 1,
    'detect_text': False,
    'detect_ship': True,
    'detect_quadrangle': True
}
train_generator = ShipGenerator(
        'train/ship_detection',
        file_dir,
        gen_type = 'train',
        ratio = 1,
        **common_args)


In [ ]:
print(train_generator.classes.keys())
my_dic = dict({"0":0,"1":0,"2":0,"3":0})

print(train_generator.size())
for i in range(train_generator.size()):
    if i%100==0:
        print(i)
    for label in train_generator.load_annotations(i)['labels']:
        my_dic[str(label)]+=1



In [ ]:
my_dic

In [ ]:
print(train_generator.load_annotations(0))
print(train_generator.load_annotations(0)['quadrangles'].astype(np.double).dtype)

In [ ]:
from utils.compute_overlap import compute_overlap
compute_overlap(train_generator.load_annotations(0)['bboxes'].astype(np.double),train_generator.load_annotations(0)['bboxes'].astype(np.double))

In [ ]:
validation_generator.size()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


In [ ]:
tf.test.is_gpu_available(train_generator.load_annotations(0)['bboxes'],train_generator.load_annotations(0)['bboxes'])

In [ ]:
from matplotlib.patches import Polygon,Rectangle
import colorsys
import random
import matplotlib.pyplot as plt
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors
    
def visualize(image, boxes,figsize=(16, 16),box_type='rect'):
        """
        desc : bbox와 함께 이미지를 그린다.
        
        -input-
        image_id : 시각화 하기를 원하는 이미지 인덱스 번호
        figsize : 이미지 크기
        """   
        fig, ax = plt.subplots(1, figsize=figsize)
        char_boxes=boxes
        char_len=len(char_boxes)
        colors = random_colors(char_len)
        print ("box channel : ", np.shape(boxes)[1])
        for i in range(char_len):
            color = colors[i]
        # Bounding box
            if not np.any(char_boxes[i]):
                # Skip this instance. Has no bbox. Likely lost in image cropping.
                continue
            
            if box_type == 'rect':
                if np.shape(boxes)[1] == 4 and len(np.shape(boxes))==3 : # 4 - vertex 
                    box = char_boxes[i]
                    y_max = max(box[0, 1], box[1, 1], box[2, 1], box[3, 1])
                    y_min = min(box[0, 1], box[1, 1], box[2, 1], box[3, 1])
                    x_max = max(box[0, 0], box[1, 0], box[2, 0], box[3, 0])
                    x_min = min(box[0, 0], box[1, 0], box[2, 0], box[3, 0])

                else : # 2 - vertex 
                    box = char_boxes[i]
                    y_max = max(box[1], box[3])
                    y_min = min(box[1], box[3])
                    x_max = max(box[0], box[2])
                    x_min = min(box[0], box[2])            
                width = (x_max-x_min)
                height = (y_max-y_min)
                print(width,height,x_min,y_min)
                p = Rectangle((x_min,y_min), width, height, linewidth=2,
                                      edgecolor=color, facecolor='none')
            elif box_type == 'quad': 
                p = Polygon(char_boxes[i], facecolor="none", edgecolor=color)
            else : 
                raise ("check the box_type")
                
            ax.add_patch(p)
        ax.imshow(image)
        

## load_image, load_annotations 확인 

In [ ]:
image = train_generator.load_image(0)
bboxes = train_generator.load_annotations(0)['bboxes']
print(bboxes[0,1])
visualize(image,bboxes)

## annotation 확인 및 시각화 

In [ ]:
image = train_generator.load_image(0)
quadboxes = train_generator.load_annotations(0)['quadrangles']
visualize(image,quadboxes,box_type='quad')

In [ ]:
if __name__ == '__main__':
    train_generator = ShipGenerator(
        'datasets/ship_detection',
        'train',
        phi=1,
        batch_size=1,
        detect_ship =True
    )
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    anchors = train_generator.anchors
    batch_inputs, batch_targets = train_generator[0]
    image = batch_inputs[0][0]
    image[..., 0] *= std[0]
    image[..., 1] *= std[1]
    image[..., 2] *= std[2]
    image[..., 0] += mean[0]
    image[..., 1] += mean[1]
    image[..., 2] += mean[2]
    image *= 255.

    regression = batch_targets[0][0]
    valid_ids = np.where(regression[:, -1] == 1)[0]
    boxes = anchors[valid_ids]
    deltas = regression[valid_ids]
    class_ids = np.argmax(batch_targets[1][0][valid_ids], axis=-1)
    mean_ = [0, 0, 0, 0]
    std_ = [0.2, 0.2, 0.2, 0.2]

    width = boxes[:, 2] - boxes[:, 0]
    height = boxes[:, 3] - boxes[:, 1]

    x1 = boxes[:, 0] + (deltas[:, 0] * std_[0] + mean_[0]) * width
    y1 = boxes[:, 1] + (deltas[:, 1] * std_[1] + mean_[1]) * height
    x2 = boxes[:, 2] + (deltas[:, 2] * std_[2] + mean_[2]) * width
    y2 = boxes[:, 3] + (deltas[:, 3] * std_[3] + mean_[3]) * height
    for x1_, y1_, x2_, y2_, class_id in zip(x1, y1, x2, y2, class_ids):
        x1_, y1_, x2_, y2_ = int(x1_), int(y1_), int(x2_), int(y2_)
        cv2.rectangle(image, (x1_, y1_), (x2_, y2_), (0, 255, 0), 2)
        class_name = train_generator.labels[class_id]
        label = class_name
        ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)
        cv2.rectangle(image, (x1_, y2_ - ret[1] - baseline), (x1_ + ret[0], y2_), (255, 255, 255), -1)
        cv2.putText(image, label, (x1_, y2_ - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.imshow('image', image.astype(np.uint8)[..., ::-1])
    cv2.waitKey(0)
    # 36864, 46080, 48384, 48960, 49104
    # if first_valid_id < 36864:
    #     stride = 8
    # elif 36864 <= first_valid_id < 46080:
    #     stride = 16
    # elif 46080 <= first_valid_id < 48384:
    #     stride = 32
    # elif 48384 <= first_valid_id < 48960:
    #     stride = 64
    # else:
    #     stride = 128
    pass